# Metrics

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import roc_curve, confusion_matrix, precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline

## 정확도의 문제점 살펴보기
**정확도(Acuuracy)** = 예측 결과가 동일한 데이터 건수 / 모든 예측 데이터 건수
- 불균형한 레이블값 분포에서 ML 모델의 선능을 판단할 경우 적합한 지표가 아님


### 1. 레이블값이 골고루 분포 돼있는 경우 

In [17]:
titanic = pd.read_csv('c:/my_web/machine_lecture/titanic_train.csv')
titanic.head()
print(titanic['Sex'].value_counts())
print()
print(titanic['Survived'].value_counts()) # 레이블 값이 고루 분포 돼있음 

male      577
female    314
Name: Sex, dtype: int64

0    549
1    342
Name: Survived, dtype: int64


titanic 데이터에서 남성(1)은 사망(0), 여성(0)은 생존(1)으로 예측하는 **이진분류기** 생성하기

In [18]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator) :
    # fit() 메소드는 아무것도 학습하지 않음 
    def fit(self, X, y = None) :
        pass
    
    # predict() 메소드는 단순히 sex feature가 남자(1)이면 사망(0), 여자(0)이면 생존(1)으로 예측
    def predict(self, X) :
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]) :
            if X['Sex'].iloc[i] == 1 :
                pred[i] = 0
            else :
                pred[i] = 1
        return pred
                

전처리 함수 생성

In [8]:
# null값 처리 함수
def fillna(df) :
    df['Age'].fillna(df['Age'].mean(), inplace = True)
    df['Cabin'].fillna('N', inplace = True)
    df['Embarked'].fillna('N', inplace = True)
    df['Fare'].fillna(0, inplace = True)
    return df

# 불필요한 속성 제거하는 함수 
def drop_features(df) :
    df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1, inplace = True)
    return df

# 인코딩 함수 
def enco_features(df) :
    df['Cabin'] = df['Cabin'].str[:1] # 맨 앞 알파벳만 가져오기
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features :
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에 생성한 함수 전부 불러오는 전처리 함수 생성
def process_features(df) :
    df = fillna(df)
    df = drop_features(df)
    df = enco_features(df)
    return df

데이터 로딩, 가공, 학습, 예측, 평가

In [11]:
# 데이터 로딩
titanic = pd.read_csv('c:/my_web/machine_lecture/titanic_train.csv')
data = titanic.drop('Survived', axis = 1)
target = titanic['Survived']

# 전처리 
data = process_features(data)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data, target,
                                                    test_size = 0.25,
                                                    random_state = 11,
                                                    stratify = target)

# 데이터 학습 
mdc = MyDummyClassifier()
mdc.fit(X_train, y_train) 

# 예측
pred = mdc.predict(X_test) 

# 평가 
print('Dummy Classifier의 정확도는 : {0: .4f}'.format(accuracy_score(y_test, pred)))

Dummy Classifier의 정확도는 :  0.8296


### 2. 레이블 값 분포가 불균형한 경우

- 데이터가 전부 0으로 분류(예측)되는 분류기 생성

In [19]:
class MyFakeClassifier(BaseEstimator) :
    def fit(self, X, y) :
        pass
    
    def predict(self, X) :
        return np.zeros((len(X), 1), dtype = bool)
    

- 사이킷런의 내장 데이터셋인 load_digits()를 이용하여 MNIST데이터 로딩

In [20]:
from sklearn.datasets import load_digits

digits = load_digits()

print(digits.data)
print('data.shape :', digits.data.shape)
print()
print(digits.target)
print('target.shape :', digits.target.shape)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
data.shape : (1797, 64)

[0 1 2 ... 8 9 8]
target.shape : (1797,)


In [25]:
df_target = pd.DataFrame(digits.target)
df_target.value_counts()

3    183
1    182
5    182
4    181
6    181
9    180
7    179
0    178
2    177
8    174
dtype: int64

In [29]:
digits.target == 7

array([False, False, False, ..., False, False, False])

- digits번호가 7번이면 **True**이고 --> **1**로 변환
- 7이 아니면 **False**이고 --> **0**으로 변환 

In [34]:
y = (digits.target == 7).astype(int)
y

array([0, 0, 0, ..., 0, 0, 0])

- 데이터 분할

In [35]:
X_train, X_test, y_train, y_test = train_test_split(digits.data, y,
                                                    random_state = 11)

- 레이블 데이터 분포도 확인 --> 불균형함을 알 수 있음 (0이 훨씬 많음)

In [37]:
print('테스트 종속변수 크기 :', y_test.shape)
print('테스트 종속변수 0과 1의 분포도 :\n', pd.Series(y_test).value_counts())

테스트 종속변수 크기 : (450,)
테스트 종속변수 0과 1의 분포도 :
 0    405
1     45
dtype: int64


- MyFakeClassifier로 학습/예측/정확도 평가해보기

In [43]:
# 학습
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)

# 예측(분류)
fake_pred = fakeclf.predict(X_test) # 데이터가 전부 0으로 예측된다. 

# 정확도 평가
print('모든 예측(분류)을 0으로 하여도 정확도는 : {: .3f}'.format(accuracy_score(y_test, fake_pred)))
# 모든 예측을 0으로 하여도 실제 데이터의 분포가 0으로 치우쳐져 있기 때문에 예측과 실제가 일치할 확률이 90퍼나 된다. 

모든 예측(분류)을 0으로 하여도 정확도는 :  0.900


#### 3. 결론 : 이진분류 종속변수의 데이터 분포가 한쪽으로 치우칠 경우 (0이나 1로 치우칠 경우) 
#### 정확도로 평가하는것은 부적절하다.
- 정확도의 한계를 보완하기 위한 평가지표를 알아보자

## Confusion Matrix (오차행렬)
오차행렬 : '이진분류'의 **예측 오류**가 얼마인지에 더불어 어떠한 유형의 예측 오류가 발생하고 있는지 함께 나타내는 지표

In [46]:
from sklearn.metrics import confusion_matrix

# 앞절의 예측 결과인 fake_pred와 실제 결과인 y_test의 Confusion Matrix 출력
confusion_matrix(y_test, fake_pred)

# TN(거짓이라(0) 예측했고 실제와 일치) --> 405개 / FP(참이라(1) 예측했는데 예측이 틀림(참으로 예측을 하나도 하지 않음 = 0개))
# FN(거짓이라(0) 예측했고 실제와 틀림(실제 1인게 45개)) / TP(참이라(1) 예측했고 예측과 일치함 (하지만 참으로 예측을 한게 없음 = 0개)) 

array([[405,   0],
       [ 45,   0]], dtype=int64)

## 정밀도(Precision)와 재현율(Recall)
- 정밀도 : 참이라고 예측한 데이터들 중 실제와 일치하는(실제도 참) 데이터 비율 --> TP / (FP + TP)
- 재현율 : 모든 참인 실제값들 중 참으로 예측한 데이터 비율 --> TP / (FN + TP)

#### 1. MyFakeclassifier의 예측 결과로 정밀도와 재현율 측정 

In [47]:
print('정밀도 :', precision_score(y_test, fake_pred)) #참이라고 예측한 데이터가 없었음  = 0
print('재현율 :', recall_score(y_test, fake_pred)) # 참인 데이터(45개)중 참으로 예측한 데이터가 없었음 = 0

# 이 두 개의 지표로 인해 정확도를 평가지표로 사용하는건 부적절 했음을 알 수 있음

정밀도 : 0.0
재현율 : 0.0


c:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 2. 업무에 따른 재현율과 정밀도의 상대적 중요도 
- 정밀도가 상대적으로 더 중요한 지표인 경우 : 실제 거짓(음성)인 데이터 예측을 참으로(양성) 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우
    - ex) 스팸메일 
- 재현율이 상대적으로 더 중요한 지표인 경우 : 실제 참인 (양성) 데이터 예측을 부정(음성)으로 잘못판단하게 되면 업무상 큰 영향이 발생하는 경우 
    - ex) 암진단, 금융사기 판별 
- **불균형한 레이블 클래스를 가지는 이진분류 모델에서는 많은 데이터 중에서 중점적으로 찾아야 하는 매우 적은 수의 결과 값에 positive를 설정해 \
1값을 부여, 그렇지 않은 경우는 negative로 0값을 부여**